# Load Libraries and Data

In [1]:
import numpy as np 
import pandas as pd 
import os, gc, re, warnings 
warnings.filterwarnings("ignore") 

In [2]:
dftr = pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/train.csv")
dftr["src"]="train"
dfte = pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/test.csv")
dfte["src"]="test"
print('Train shape:',dftr.shape,'Test shape:',dfte.shape,'Test columns:',dfte.columns)
df = pd.concat([dftr,dfte],ignore_index=True)

dftr.head()

Train shape: (3911, 9) Test shape: (3, 3) Test columns: Index(['text_id', 'full_text', 'src'], dtype='object')


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,src
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,train
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,train
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,train
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,train
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,train


In [3]:
target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions',]

# Make 25 Stratified Folds!

In [4]:
import sys
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
FOLDS = 25
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
for i,(train_index, val_index) in enumerate(skf.split(dftr,dftr[target_cols])):
    dftr.loc[val_index,'FOLD'] = i
print('Train samples per fold:')
dftr.FOLD.value_counts()

Train samples per fold:


11.0    157
21.0    157
1.0     157
7.0     157
20.0    157
14.0    157
19.0    157
12.0    157
23.0    157
3.0     157
6.0     157
5.0     156
13.0    156
22.0    156
0.0     156
15.0    156
24.0    156
17.0    156
9.0     156
8.0     156
4.0     156
2.0     156
18.0    156
10.0    156
16.0    156
Name: FOLD, dtype: int64

# Generate Embeddings

In [5]:
from transformers import AutoModel,AutoTokenizer
import torch
import torch.nn.functional as F
from tqdm import tqdm

In [6]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state.detach().cpu()
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

In [7]:
BATCH_SIZE = 4

class EmbedDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.df = df.reset_index(drop=True)
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        text = self.df.loc[idx,"full_text"]
        tokens = tokenizer(
                text,
                None,
                add_special_tokens=True,
                padding='max_length',
                truncation=True,
                max_length=MAX_LEN,return_tensors="pt")
        tokens = {k:v.squeeze(0) for k,v in tokens.items()}
        return tokens

ds_tr = EmbedDataset(dftr)
embed_dataloader_tr = torch.utils.data.DataLoader(ds_tr,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False)
ds_te = EmbedDataset(dfte)
embed_dataloader_te = torch.utils.data.DataLoader(ds_te,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False)

# Extract Embeddings

In [8]:
tokenizer = None
MAX_LEN = 512

def get_embeddings(MODEL_NM='', TOK_NM = '', MAX=512, BATCH_SIZE=4, verbose=True):
    global tokenizer, MAX_LEN
    DEVICE="cuda"
    model = AutoModel.from_pretrained( MODEL_NM )
    tokenizer = AutoTokenizer.from_pretrained( MODEL_NM )
    MAX_LEN = MAX
    
    model = model.to(DEVICE)
    model.eval()
    all_train_text_feats = []
    for batch in tqdm(embed_dataloader_tr,total=len(embed_dataloader_tr)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        all_train_text_feats.extend(sentence_embeddings)
    all_train_text_feats = np.array(all_train_text_feats)
    if verbose:
        print('Train embeddings shape',all_train_text_feats.shape)
        
    te_text_feats = []
    for batch in tqdm(embed_dataloader_te,total=len(embed_dataloader_te)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        te_text_feats.extend(sentence_embeddings)
    te_text_feats = np.array(te_text_feats)
    if verbose:
        print('Test embeddings shape',te_text_feats.shape)
        
    return all_train_text_feats, te_text_feats

In [9]:
import os
path = "fb3_embeddings"
isExist = os.path.exists(path)
if not isExist:
    os.makedirs(path)
    print("The new directory is created!")

The new directory is created!


### Get Base Embeddings all-roberta-large-v1

In [10]:
MODEL_NM = "facebook/bart-large"
all_train_text_feats, te_text_feats = get_embeddings(MODEL_NM) 

with open(f"fb3_embeddings/fb_bart_large.npy", 'wb') as f:
    np.save(f, all_train_text_feats)

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/971M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

100%|██████████| 978/978 [05:13<00:00,  3.12it/s]


Train embeddings shape (3911, 1024)


100%|██████████| 1/1 [00:00<00:00,  4.47it/s]

Test embeddings shape (3, 1024)


In [11]:
all_train_text_feats = np.concatenate([all_train_text_feats,],axis=1)
te_text_feats = np.concatenate([te_text_feats,],axis=1)
gc.collect()

print('Our concatenated embeddings have shape', all_train_text_feats.shape)

Our concatenated embeddings have shape (3911, 1024)


# Train RAPIDS cuML SVR
Documentation for RAPIDS SVM is [here][1]

[1]: https://docs.rapids.ai/api/cuml/stable/api.html#support-vector-machines

In [12]:
from cuml.svm import SVR
import cuml
print('RAPIDS version',cuml.__version__)

RAPIDS version 21.10.02


In [13]:
from sklearn.metrics import mean_squared_error

preds = []
scores = []
def comp_score(y_true,y_pred):
    rmse_scores = []
    for i in range(len(target_cols)):
        rmse_scores.append(np.sqrt(mean_squared_error(y_true[:,i],y_pred[:,i])))
    return np.mean(rmse_scores)


oof_df = pd.DataFrame()
#for fold in tqdm(range(FOLDS),total=FOLDS):
for fold in range(FOLDS):
    print('#'*25) 
    print('### Fold',fold+1) 
    print('#'*25) 
    
    dftr_ = dftr[dftr["FOLD"]!=fold] 
    dfev_ = dftr[dftr["FOLD"]==fold] 
    
    tr_text_feats = all_train_text_feats[list(dftr_.index),:]
    ev_text_feats = all_train_text_feats[list(dfev_.index),:]
    
    ev_preds = np.zeros((len(ev_text_feats),6))
    test_preds = np.zeros((len(te_text_feats),6))
    for i,t in enumerate(target_cols):
        print(t,', ',end='')
        clf = SVR(C=1)
        clf.fit(tr_text_feats, dftr_[t].values)
        ev_preds[:,i] = clf.predict(ev_text_feats)
        test_preds[:,i] = clf.predict(te_text_feats)
    print() 
    score = comp_score(dfev_[target_cols].values,ev_preds)
    scores.append(score)
    print("Fold : {} RSME score: {}".format(fold,score))
    
    dfev_[[f"pred_{c}" for c in target_cols]] = ev_preds
    oof_df = pd.concat([oof_df, dfev_])
    preds.append(test_preds)
    
oof_df = oof_df.reset_index(drop=True)
oof_df.to_csv(f"fb3_embeddings/fb_bart_large_512_oof.csv")    

print('#'*25)
print('Overall CV RSME =',np.mean(scores))

#########################
### Fold 1
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 0 RSME score: 0.4674734393036422
#########################
### Fold 2
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 1 RSME score: 0.45977760882071594
#########################
### Fold 3
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 2 RSME score: 0.47532701883274625
#########################
### Fold 4
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 3 RSME score: 0.4578513058101757
#########################
### Fold 5
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 4 RSME score: 0.45620286581210984
#########################
### Fold 6
#########################
cohesion , syntax , vocabulary , phraseology , grammar , co

### Get Base Embeddings flax-sentence-embeddings/all_datasets_v3_roberta-large

## 

In [14]:
MODEL_NM = "flax-sentence-embeddings/all_datasets_v3_roberta-large"
all_train_text_feats, te_text_feats = get_embeddings(MODEL_NM) 

with open(f"fb3_embeddings/ad_v3_roberta_large.npy", 'wb') as f:
    np.save(f, all_train_text_feats)

Downloading:   0%|          | 0.00/650 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/328 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

100%|██████████| 978/978 [04:20<00:00,  3.75it/s]


Train embeddings shape (3911, 1024)


100%|██████████| 1/1 [00:00<00:00,  5.34it/s]


Test embeddings shape (3, 1024)


In [15]:
all_train_text_feats = np.concatenate([all_train_text_feats,],axis=1)
te_text_feats = np.concatenate([te_text_feats,],axis=1)
gc.collect()

print('Our concatenated embeddings have shape', all_train_text_feats.shape)

Our concatenated embeddings have shape (3911, 1024)


In [16]:
from sklearn.metrics import mean_squared_error

preds = []
scores = []
def comp_score(y_true,y_pred):
    rmse_scores = []
    for i in range(len(target_cols)):
        rmse_scores.append(np.sqrt(mean_squared_error(y_true[:,i],y_pred[:,i])))
    return np.mean(rmse_scores)


oof_df = pd.DataFrame()
#for fold in tqdm(range(FOLDS),total=FOLDS):
for fold in range(FOLDS):
    print('#'*25) 
    print('### Fold',fold+1) 
    print('#'*25) 
    
    dftr_ = dftr[dftr["FOLD"]!=fold] 
    dfev_ = dftr[dftr["FOLD"]==fold] 
    
    tr_text_feats = all_train_text_feats[list(dftr_.index),:]
    ev_text_feats = all_train_text_feats[list(dfev_.index),:]
    
    ev_preds = np.zeros((len(ev_text_feats),6))
    test_preds = np.zeros((len(te_text_feats),6))
    for i,t in enumerate(target_cols):
        print(t,', ',end='')
        clf = SVR(C=1)
        clf.fit(tr_text_feats, dftr_[t].values)
        ev_preds[:,i] = clf.predict(ev_text_feats)
        test_preds[:,i] = clf.predict(te_text_feats)
    print() 
    score = comp_score(dfev_[target_cols].values,ev_preds)
    scores.append(score)
    print("Fold : {} RSME score: {}".format(fold,score))
    
    dfev_[[f"pred_{c}" for c in target_cols]] = ev_preds
    oof_df = pd.concat([oof_df, dfev_])
    preds.append(test_preds)
    
oof_df = oof_df.reset_index(drop=True)
oof_df.to_csv(f"fb3_embeddings/ad_v3_roberta_large.csv")    

print('#'*25)
print('Overall CV RSME =',np.mean(scores))

#########################
### Fold 1
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 0 RSME score: 0.49740754752263183
#########################
### Fold 2
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 1 RSME score: 0.4986975988622538
#########################
### Fold 3
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 2 RSME score: 0.5163395057448988
#########################
### Fold 4
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 3 RSME score: 0.4933252589668251
#########################
### Fold 5
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 4 RSME score: 0.5139417091124985
#########################
### Fold 6
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conv

### Get Base Embeddings facebook/bart-large-mnli

In [17]:
MODEL_NM = "facebook/bart-large-mnli"
all_train_text_feats, te_text_feats = get_embeddings(MODEL_NM) 

with open(f"fb3_embeddings/fb_bart_large_mnli.npy", 'wb') as f:
    np.save(f, all_train_text_feats)

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartModel: ['classification_head.out_proj.weight', 'classification_head.out_proj.bias', 'classification_head.dense.bias', 'classification_head.dense.weight']
- This IS expected if you are initializing BartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

100%|██████████| 978/978 [05:12<00:00,  3.13it/s]


Train embeddings shape (3911, 1024)


100%|██████████| 1/1 [00:00<00:00,  4.49it/s]

Test embeddings shape (3, 1024)


In [18]:
all_train_text_feats = np.concatenate([all_train_text_feats,],axis=1)
te_text_feats = np.concatenate([te_text_feats,],axis=1)
gc.collect()

print('Our concatenated embeddings have shape', all_train_text_feats.shape)

Our concatenated embeddings have shape (3911, 1024)


In [19]:
from sklearn.metrics import mean_squared_error

preds = []
scores = []
def comp_score(y_true,y_pred):
    rmse_scores = []
    for i in range(len(target_cols)):
        rmse_scores.append(np.sqrt(mean_squared_error(y_true[:,i],y_pred[:,i])))
    return np.mean(rmse_scores)


oof_df = pd.DataFrame()
#for fold in tqdm(range(FOLDS),total=FOLDS):
for fold in range(FOLDS):
    print('#'*25) 
    print('### Fold',fold+1) 
    print('#'*25) 
    
    dftr_ = dftr[dftr["FOLD"]!=fold] 
    dfev_ = dftr[dftr["FOLD"]==fold] 
    
    tr_text_feats = all_train_text_feats[list(dftr_.index),:]
    ev_text_feats = all_train_text_feats[list(dfev_.index),:]
    
    ev_preds = np.zeros((len(ev_text_feats),6))
    test_preds = np.zeros((len(te_text_feats),6))
    for i,t in enumerate(target_cols):
        print(t,', ',end='')
        clf = SVR(C=1)
        clf.fit(tr_text_feats, dftr_[t].values)
        ev_preds[:,i] = clf.predict(ev_text_feats)
        test_preds[:,i] = clf.predict(te_text_feats)
    print() 
    score = comp_score(dfev_[target_cols].values,ev_preds)
    scores.append(score)
    print("Fold : {} RSME score: {}".format(fold,score))
    
    dfev_[[f"pred_{c}" for c in target_cols]] = ev_preds
    oof_df = pd.concat([oof_df, dfev_])
    preds.append(test_preds)
    
oof_df = oof_df.reset_index(drop=True)
oof_df.to_csv(f"fb3_embeddings/fb_bart_large_mnli.csv")    

print('#'*25)
print('Overall CV RSME =',np.mean(scores))

#########################
### Fold 1
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 0 RSME score: 0.4816144643446554
#########################
### Fold 2
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 1 RSME score: 0.46865929707304405
#########################
### Fold 3
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 2 RSME score: 0.47644566523631227
#########################
### Fold 4
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 3 RSME score: 0.4695211421476753
#########################
### Fold 5
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 4 RSME score: 0.466210265842798
#########################
### Fold 6
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conv